In [6]:
import base64
import json
import base64
import json
from Cryptodome.Cipher import AES
from phpserialize import loads
KEY = b'EDnpXl5oxi9+XHjTUbTwMg98jTeCt4tnJx5LaUtanME='

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pipreqs` not found.


In [44]:
def decrypt(laravelEncrypedStringBase64, laravelAppKeyBase64):
    # Decode from base64 Laravel encrypted string
    dataJson = base64.b64decode(laravelEncrypedStringBase64)
    # Load JSON
    data = json.loads(dataJson)
    # Extract actual encrypted message from JSON (other parts are IV and Signature)
    value =  base64.b64decode(data['value'])
    # Extract Initialization Vector from JSON (required to create an AES decypher)
    iv = base64.b64decode(data['iv'])
    # Decode 
    key = base64.b64decode(laravelAppKeyBase64)  # Laravel KEY comes base64Encoded from .env!
    # Create an AES decypher
    decrypter = aesDecrypterCBC(iv, key)
    # Finally decypher the message
    decriptedSerializedMessage = decrypter.decrypt(value)
    # deserialize message
    try :
        # Attempt to deserialize message incase it was created in Laravel with Crypt::encrypt('Hello world.');
        decriptedMessage = unserialize(decriptedSerializedMessage)
        return str(decriptedMessage)
    except:
        raise Exception("Check you cyphered strings in Laravel using Crypt::encrypt() and NOT Crypt::encryptString()")

def aesDecrypterCBC(iv, _key):
    decrypterAES_CBC = AES.new(key=_key,mode=AES.MODE_CBC,IV=iv)
    return decrypterAES_CBC

def unserialize(serialized):
    return loads(serialized)

def decrypt_message(raw_message):
    decoded_message = base64.b64decode(raw_message)
    filtered_message = b''.join(decoded_message.split(b",",3)[:3]).replace(b'""',b'","')
    if len(filtered_message) != len(decoded_message):
        filtered_message = filtered_message + b'}'
    message = base64.b64encode(filtered_message)
    key = KEY
    message = decrypt(message,key)
    return_value = base64.b64encode(message.encode("utf8"))
    return return_value

In [45]:
##For testing purposes
raw_message = b"eyJpdiI6ImR4eC9pVEV1WERRMlZYaGxyOE5rMVE9PSIsInZhbHVlIjoidnlIMmRZam9sbG9QUmlaWmdua2hJWG5kdHBSdDZXdWc4d3FkKy95d0U5OXZIM2QrWUt2ZDVlRjEraHEyRDlQRGlldW01VWZKaEhncTZ0Z3NuV2hTdW84S2dMaFNJVFc4eFRqYzJlclk0dnM9IiwibWFjIjoiYmY3YWZiYmM1NDU1MDk5YWFkYWY2MDY4OTA4MDZlNmQ3NDJkZWYyMjFmNzFhOGQwNDQ4MzJmNjAxYzJlMzExZCJ9"
decrypt_message(raw_message)

b'eyJpdiI6ImR4eC9pVEV1WERRMlZYaGxyOE5rMVE9PSIsInZhbHVlIjoidnlIMmRZam9sbG9QUmlaWmdua2hJWG5kdHBSdDZXdWc4d3FkKy95d0U5OXZIM2QrWUt2ZDVlRjEraHEyRDlQRGlldW01VWZKaEhncTZ0Z3NuV2hTdW84S2dMaFNJVFc4eFRqYzJlclk0dnM9IiwibWFjIjoiYmY3YWZiYmM1NDU1MDk5YWFkYWY2MDY4OTA4MDZlNmQ3NDJkZWYyMjFmNzFhOGQwNDQ4MzJmNjAxYzJlMzExZCJ9'
b'{"iv":"dxx/iTEuXDQ2VXhlr8Nk1Q==","value":"vyH2dYjolloPRiZZgnkhIXndtpRt6Wug8wqd+/ywE99vH3d+YKvd5eF1+hq2D9PDieum5UfJhHgq6tgsnWhSuo8KgLhSITW8xTjc2erY4vs=","mac":"bf7afbbc5455099aadaf606890806e6d742def221f71a8d044832f601c2e311d"}'
b'{"iv":"dxx/iTEuXDQ2VXhlr8Nk1Q==","value":"vyH2dYjolloPRiZZgnkhIXndtpRt6Wug8wqd+/ywE99vH3d+YKvd5eF1+hq2D9PDieum5UfJhHgq6tgsnWhSuo8KgLhSITW8xTjc2erY4vs=","mac":"bf7afbbc5455099aadaf606890806e6d742def221f71a8d044832f601c2e311d"}'
b'eyJpdiI6ImR4eC9pVEV1WERRMlZYaGxyOE5rMVE9PSIsInZhbHVlIjoidnlIMmRZam9sbG9QUmlaWmdua2hJWG5kdHBSdDZXdWc4d3FkKy95d0U5OXZIM2QrWUt2ZDVlRjEraHEyRDlQRGlldW01VWZKaEhncTZ0Z3NuV2hTdW84S2dMaFNJVFc4eFRqYzJlclk0dnM9IiwibWFjIjoiYmY3YWZiYmM1NDU1MDk5YWFk

b'YidPOjg6InN0ZENsYXNzIjoyOntzOjg6InJlbGF0aW9uIjtpOjEzNjY7czo4OiJkb2N1bWVudCI7czoxOiJQIjt9Jw=='

# 